In [1]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

# np_ar = np.asarray(df)
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [ ]:
# print(df['Planting date'][[np.random.randn]])
# print(df.columns)
# print(df.Temperature.describe())
# import random
# for i in range(0, 500):
#     print(random.choice(df['Planting date']))

In [2]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
variety_dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, variety_dummies], axis=1)


In [3]:
# OPTIONAL VARIETY DISTRIBUTION ANALYSIS

print(variety_dummies.sum().describe())
print(np.sort(variety_dummies.sum()))
for idx, cl in enumerate(variety_dummies.sum()):
    print(variety_dummies.columns[idx], cl)

count     174.000000
mean      471.471264
std       642.755886
min        38.000000
25%       111.250000
50%       215.000000
75%       560.000000
max      3849.000000
dtype: float64
[  38   40   45   46   56   57   66   70   71   71   73   75   76   76
   76   77   80   80   81   81   82   82   87   88   89   93   93   94
   94   95   96   96   96   97   98   98  103  104  106  106  106  107
  110  111  112  113  114  114  115  115  118  124  125  125  127  131
  131  132  134  135  145  145  146  150  150  152  154  154  159  161
  163  163  167  170  179  179  180  180  186  186  187  188  192  193
  198  211  213  217  218  218  220  222  223  230  231  237  245  249
  258  271  273  279  281  283  283  291  299  303  304  304  310  311
  312  319  319  322  325  325  330  337  345  345  369  389  410  439
  442  464  543  545  565  576  581  596  604  611  622  628  673  679
  723  729  738  788  793  819  834  835  872 1004 1040 1095 1114 1133
 1212 1226 1236 1246 1269 1371 1445 

In [4]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [5]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [ ]:
print(df.columns)

In [ ]:
pd.get_dummies(df['Season']).sum()

In [ ]:
print(df.columns)
print(df.isnull().sum())

In [ ]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [ ]:
np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

In [ ]:
#THIS IS A VISUALIZATION FOR LATITUDE AND LONGITUDE CLUSTERING

cent = kmeans.cluster_centers_
clust_labels = kmeans.labels_
means = pd.DataFrame(clust_labels)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(df['Latitude'],df['Longitude'],
                     c=means[0], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
plt.colorbar(scatter)

In [ ]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
binarized = lb.fit(df.Variety)
print(binarized)
df.Variety = pd.Series(binarized.transform(df.Variety))
print(binarized.transform(df.Variety).reshape(1,-1))

In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [6]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date', 'Season']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

The final dataframe has columns:  Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay',
       ...
       'V156763', 'V156774', 'V156783', 'V156786', 'V156797', 'V156806',
       'V156807', 'May', 'June', 'YieldBucket'],
      dtype='object', length=190)


In [ ]:
print(pd.get_dummies(df.YieldBucket).sum())

In [7]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

We expect to be 0 nan values and there actually are 0 nan values

Yield              0
Year               0
Temperature        0
Precipitation      0
Solar Radiation    0
Soil class         0
CEC                0
Organic matter     0
pH                 0
Clay               0
Silt               0
Sand               0
Area               0
V000016            0
V000017            0
V000018            0
V000023            0
V000024            0
V000025            0
V000030            0
V000032            0
V000034            0
V000036            0
V000039            0
V000047            0
V000050            0
V000051            0
V000058            0
V000060            0
V000062            0
                  ..
V152320            0
V152322            0
V152440            0
V152734            0
V152779            0
V155180            0
V155820            0
V155842            0
V155843            0
V155918            0
V156247            0
V156305            0
V156314            0
V156367   

In [9]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt',
       ...
       'V156642', 'V156763', 'V156774', 'V156783', 'V156786', 'V156797',
       'V156806', 'V156807', 'May', 'June'],
      dtype='object', length=188)


In [10]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (8008, 188) 
y_train shape: (8008,)


In [11]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [12]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)


RMSE Error:  7.239297343409036
Average Error details:
 count    4005.000000
mean       11.263935
std        76.715278
min         0.001094
25%         3.373547
50%         7.328156
75%        13.316691
max      4818.201404
Name: Yield, dtype: float64


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


75383     7.850797
72075    18.005133
957       7.524621
24427     6.941080
43478    15.501578
50475     0.319069
74316     9.868616
68495     2.063393
75949     0.831735
20334     9.710521
1410      8.379913
32618     9.900456
3276      7.586573
21243     8.572071
55467     6.303217
50480     1.002185
14173     0.558369
28926    17.029958
1705      0.230575
33565     5.705147
61522     7.241548
25348    24.467656
75985     3.659267
33665     4.004196
13205     1.260624
15319     6.113482
69101    22.226414
80902     6.476884
53777     0.204304
64899     6.334625
           ...    
27685    22.492898
68474     5.375457
5256      9.286771
72737    26.981277
54543     4.731919
10836     1.239945
60139    12.067514
35322     0.818179
57693     9.014146
30743    20.807053
14530    17.087985
37800    13.938172
18228     3.355582
80146     0.733234
14477     2.123516
28425    12.426181
80659     0.159748
43424    29.130353
69267    13.857436
29461     3.835108
41730     6.692658
11385     0.

In [ ]:
pd.get_dummies(df['Plant date']).sum()

In [13]:
# GET OUTPUT OF FEATURE IMPORTANCE
def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

V103332 has importance:  0.0
V113476 has importance:  1.5346996237694667e-08
V103193 has importance:  6.751438236880648e-06
V000036 has importance:  7.352859006031621e-06
V156783 has importance:  1.0313255638034102e-05
V000017 has importance:  1.2040113048147601e-05
V110768 has importance:  1.911830117895028e-05
V121097 has importance:  1.952195485178907e-05
V156763 has importance:  2.106775559170475e-05
V152102 has importance:  2.2602239151141237e-05
V140091 has importance:  2.3634600256927413e-05
V151340 has importance:  2.6488185870112705e-05
V151334 has importance:  2.966461945009675e-05
V103303 has importance:  3.202299694593789e-05
V103466 has importance:  3.7749725710246e-05
V000030 has importance:  4.0477946344083594e-05
V103266 has importance:  4.2238307763988933e-05
V114951 has importance:  4.728094660253888e-05
V152312 has importance:  4.968976218700339e-05
V139094 has importance:  5.27777844518859e-05
V151407 has importance:  5.574482439120131e-05
V113424 has importance:  5

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(errors)
#     print(errors.describe())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))